In [55]:
import utils, sys, re, functools

data_ex= [
    "1abc2",
    "pqr3stu8vwx",
    "a1b2c3d4e5f",
    "treb7uchet",
]
data = utils.readfile("data/d01.txt")

def day01(data):
    r = []
    for line in data:
        title = re.findall('([0-9])', line)
        r.append(int(str(title[0]+""+title[-1])))
    return sum(r)
        
assert day01(data_ex) == 142
print(day01(data))

numbers = {
    "zero":0,
    "one":1,
    "two":2,
    "three":3,
    "four":4,
    "five":5,
    "six":6,
    "seven":7,
    "eight":8,
    "nine":9
}
def day01_B(data):
    r = []
    for line in data:
        title = re.findall('(?=([0-9]|one|two|three|four|five|six|seven|eight|nine))', line, flags=re.IGNORECASE)
        print(line, ">", title)
        v1 = int(numbers[title[0]]) if title[0] in numbers else int(title[0])
        v2 = int(numbers[title[-1]]) if title[-1] in numbers else int(title[-1])
        s = int(str(v1)+""+str(v2))
        print(s)
        r.append(s)
    return sum(r)

data_ex2 = [
    "two1nine",
    "eightwothree",
    "abcone2threexyz",
    "xtwone3four",
    "4nineeightseven2",
    "zoneight234",
    "7pqrstsixteen"
]
assert day01_B(data_ex2) == 281

assert day01_B(["eighthree","sevenine"]) == 83 + 79

print(day01_B(data))

data/d01.txt
53386
two1nine > ['two', '1', 'nine']
29
eightwothree > ['eight', 'two', 'three']
83
abcone2threexyz > ['one', '2', 'three']
13
xtwone3four > ['two', 'one', '3', 'four']
24
4nineeightseven2 > ['4', 'nine', 'eight', 'seven', '2']
42
zoneight234 > ['one', 'eight', '2', '3', '4']
14
7pqrstsixteen > ['7', 'six']
76
eighthree > ['eight', 'three']
83
sevenine > ['seven', 'nine']
79
ckmb52fldxkseven3fkjgcbzmnr7 > ['5', '2', 'seven', '3', '7']
57
gckhqpb6twoqnjxqplthree2fourkspnsnzxlz1 > ['6', 'two', 'three', '2', 'four', '1']
61
2onetwocrgbqm7 > ['2', 'one', 'two', '7']
27
frkh2nineqmqxrvdsevenfive > ['2', 'nine', 'seven', 'five']
25
four9two > ['four', '9', 'two']
42
six7sixqrdfive3twonehsk > ['six', '7', 'six', 'five', '3', 'two', 'one']
61
xkvsone2 > ['one', '2']
12
one65 > ['one', '6', '5']
15
rggxsff1seven > ['1', 'seven']
17
djbcgrrtqdshpqqzj43rgcr > ['4', '3']
43
br89fivetwoqggnxjfourtl3 > ['8', '9', 'five', 'two', 'four', '3']
83
zoneight47five5sixjxd74 > ['one', 'eight',

7rjvqdvgxxmb > ['7']
77
sixbkdpftd6 > ['six', '6']
66
699 > ['6', '9', '9']
69
rflvscthxb9one9jtfqd > ['9', 'one', '9']
99
sixtwo2hsclsg > ['six', 'two', '2']
62
8nzgpt7eightfour6zhjxqjmrq53 > ['8', '7', 'eight', 'four', '6', '5', '3']
83
pddnfour1one749five > ['four', '1', 'one', '7', '4', '9', 'five']
45
gbbprvrq23seven77zssgktwo > ['2', '3', 'seven', '7', '7', 'two']
22
ppc2flmdjrp7twoonerjhtlv > ['2', '7', 'two', 'one']
21
vlltgb8seveneight5clqsthree6 > ['8', 'seven', 'eight', '5', 'three', '6']
86
onetwobtgdkkxv1five6lcfvkhm9three > ['one', 'two', '1', 'five', '6', '9', 'three']
13
two8threesixsix > ['two', '8', 'three', 'six', 'six']
26
5threeeight > ['5', 'three', 'eight']
58
onespeight9twonexpr > ['one', 'eight', '9', 'two', 'one']
11
pzfourseven8five9five > ['four', 'seven', '8', 'five', '9', 'five']
45
9mbdbphhrfeight > ['9', 'eight']
98
fournt14 > ['four', '1', '4']
44
jlcb2 > ['2']
22
three3eightnfj2 > ['three', '3', 'eight', '2']
32
6four2fgsndlslkr > ['6', 'four', '2']
62